In [20]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score,f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

In [3]:
dataset_train = 'Dataset/train'
dataset_test = 'Dataset/test'

In [4]:
img_height, img_width = 150, 150
batch_size = 32

In [5]:
datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values to [0, 1]
    validation_split=0.2,      # Split dataset into training (80%) and validation (20%)
    rotation_range=20,         # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,     # Randomly shift the width of images
    height_shift_range=0.2,    # Randomly shift the height of images
    shear_range=0.2,           # Apply shear transformations
    zoom_range=0.2,            # Randomly zoom into images
    horizontal_flip=True,      # Randomly flip images horizontally
    fill_mode='nearest'        # Fill empty pixels using the nearest available pixel
)

In [12]:
train_generator = datagen.flow_from_directory(
    dataset_train,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
    subset='training'       # Specify 'training' for the training dataset
)

Found 4187 images belonging to 2 classes.


In [13]:
validation_generator = datagen.flow_from_directory(
    dataset_train,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
    subset='validation'     # Specify 'validation' for the validation dataset
)

Found 1045 images belonging to 2 classes.


In [14]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Add dropout for regularization
    Dense(256, activation='relu'),
    Dropout(0.5),  # Add dropout for regularization
    Dense(1, activation='sigmoid')  # Binary classification, so use 'sigmoid' activation
])


2023-09-26 12:02:47.650503: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-26 12:02:47.818633: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-26 12:02:47.818884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-26 12:02:47.819367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [15]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary classification loss
              metrics=['accuracy'])

In [16]:
epochs = 20  # Adjust the number of epochs as needed
model.fit(train_generator, validation_data=validation_generator, epochs=epochs)

Epoch 1/20


2023-09-26 12:02:57.686848: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2023-09-26 12:02:58.382012: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-26 12:02:58.711283: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-26 12:02:58.806640: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f0367225340 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-26 12:02:58.806683: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-09-26 12:02:58.810930: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-26 12:

131/131 [==============================] - 48s 335ms/step - loss: 0.5519 - accuracy: 0.7440 - val_loss: 0.6139 - val_accuracy: 0.6545
Epoch 2/20
131/131 [==============================] - 41s 316ms/step - loss: 0.4386 - accuracy: 0.7736 - val_loss: 0.4772 - val_accuracy: 0.7388
Epoch 3/20
131/131 [==============================] - 39s 301ms/step - loss: 0.3581 - accuracy: 0.8335 - val_loss: 0.3723 - val_accuracy: 0.8115
Epoch 4/20
131/131 [==============================] - 40s 306ms/step - loss: 0.2932 - accuracy: 0.8746 - val_loss: 0.3009 - val_accuracy: 0.8689
Epoch 5/20
131/131 [==============================] - 39s 300ms/step - loss: 0.2773 - accuracy: 0.8791 - val_loss: 0.3002 - val_accuracy: 0.8622
Epoch 6/20
131/131 [==============================] - 41s 312ms/step - loss: 0.2640 - accuracy: 0.8846 - val_loss: 0.2969 - val_accuracy: 0.8718
Epoch 7/20
131/131 [==============================] - 40s 306ms/step - loss: 0.2497 - accuracy: 0.8942 - val_loss: 0.2806 - val_accuracy: 0.8

In [17]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    dataset_test,  # Replace with the path to your test dataset directory
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
    shuffle=False  # Do not shuffle the data for evaluation
)
test_preds = model.predict(test_generator)
predicted_classes = np.round(test_preds).flatten()  # Convert probabilities to binary predictions

true_classes = test_generator.classes

Found 624 images belonging to 2 classes.
20/20 [==============================] - 3s 135ms/step


In [22]:
accuracy = accuracy_score(true_classes, predicted_classes)
clr = classification_report(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)
print(f"Accuracy: {accuracy}")
print(f"F1 Score:{f1}")
print(f"Classification Report:\n{clr}")

Accuracy: 0.8862179487179487
F1 Score:0.9109159347553325
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.81      0.84       234
           1       0.89      0.93      0.91       390

    accuracy                           0.89       624
   macro avg       0.88      0.87      0.88       624
weighted avg       0.89      0.89      0.89       624

